In [ ]:
import json
import os
from symspellpy import SymSpell, Verbosity
from ultralytics import YOLO

In [ ]:
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
try:
    sym_spell.load_dictionary("ru_full.txt", term_index=0, count_index=1, encoding='utf-8')
except Exception as e:
    print(f"Ошибка загрузки словаря: {e}")
    exit(1)

In [ ]:
try:
    model = YOLO("best.pt")
except Exception as e:
    print(f"Ошибка загрузки модели YOLO: {e}")
    exit(1)

In [ ]:
def correct_text(text):
    suggestions = sym_spell.lookup(text, Verbosity.CLOSEST, max_edit_distance=2)
    return suggestions[0].term if suggestions else text

In [ ]:
def extract_text_from_webres(webres_path):
    try:
        with open(webres_path, 'r', encoding='utf-8', errors='replace') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Ошибка чтения {webres_path}: {e}")
        return []

    text_boxes = []

    def parse_element(element):
        if isinstance(element, dict):
            if 'languages' in element:
                for lang in element['languages']:
                    if lang.get('lang') == 'rus':
                        for text_item in lang.get('texts', []):
                            text = text_item.get('text', '').strip()
                            if text:
                                corrected = correct_text(text)
                                box = {
                                    'x': element.get('x', 0),
                                    'y': element.get('y', 0),
                                    'w': element.get('w', 0),
                                    'h': element.get('h', 0),
                                    'text': corrected
                                }
                                text_boxes.append(box)
            for value in element.values():
                parse_element(value)
        elif isinstance(element, list):
            for item in element:
                parse_element(item)

    parse_element(data)
    return text_boxes

In [ ]:
def get_yolo_boxes(image_path):
    try:
        results = model(image_path)
    except Exception as e:
        print(f"Ошибка обработки изображения {image_path}: {e}")
        return []

    yolo_boxes = []
    print(f"\nДетекция для {image_path}:")

    for result in results:
        for box in result.boxes:
            xyxy = box.xyxy[0].tolist()
            cls_id = int(box.cls)
            conf = box.conf.item()
            print(f"  Класс: {cls_id}, Метка: {model.names[cls_id]}, Conf: {conf:.2f}, BBox: {xyxy}")

            yolo_boxes.append({
                'x1': xyxy[0],
                'y1': xyxy[1],
                'x2': xyxy[2],
                'y2': xyxy[3],
                'label': cls_id
            })
    return yolo_boxes


In [ ]:
def match_text_to_tasks(webres_boxes, yolo_boxes):
    task_data = []
    print("\nСопоставление боксов:")

    for yolo_box in yolo_boxes:
        task_num = yolo_box['label']
        task_key = f"Task_2{task_num + 2}"
        matched_texts = []

        for webres_box in webres_boxes:
            text_center_x = webres_box['x'] + webres_box['w'] / 2
            text_center_y = webres_box['y'] + webres_box['h'] / 2

            if (yolo_box['x1'] <= text_center_x <= yolo_box['x2'] and
                    yolo_box['y1'] <= text_center_y <= yolo_box['y2']):
                matched_texts.append(webres_box['text'])
                print(f"  Найдено совпадение: {task_key} -> '{webres_box['text']}'")

        if matched_texts:
            combined_text = " ".join(matched_texts)
            task_data.append({
                'task': task_key,
                'text': combined_text
            })

    return task_data


In [ ]:
def find_pairs(webres_dir, images_dir):
    pairs = []
    try:
        webres_files = [f for f in os.listdir(webres_dir) if f.endswith('.webRes')]
    except Exception as e:
        print(f"Ошибка чтения директории {webres_dir}: {e}")
        return pairs

    for webres_file in webres_files:
        base_name = os.path.splitext(webres_file)[0].split('__')[0]
        image_file = f"{base_name}.png"
        image_path = os.path.join(images_dir, image_file)

        if os.path.exists(image_path):
            pairs.append((
                os.path.join(webres_dir, webres_file),
                image_path,
                webres_file
            ))
        else:
            print(f"Предупреждение: Не найден .png файл для {webres_file}")

    print(f"Найдено {len(pairs)} пар файлов")
    return pairs


In [ ]:
def main(webres_dir, images_dir, output_file="results.txt"):
    pairs = find_pairs(webres_dir, images_dir)
    all_results = []

    for webres_path, image_path, source_file in pairs:
        print(f"\nОбработка: {source_file} + {os.path.basename(image_path)}")
        try:
            webres_boxes = extract_text_from_webres(webres_path)
            yolo_boxes = get_yolo_boxes(image_path)
            task_data = match_text_to_tasks(webres_boxes, yolo_boxes, source_file)
            all_results.extend(task_data)

            print(f"Найдено задач в текущей паре: {len(task_data)}")
        except Exception as e:
            print(f"Ошибка обработки пары: {e}")

    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            for result in sorted(all_results, key=lambda x: (x['task'])):
                f.write(f"{result['task']}: \"{result['text']}\"\n")

        print("\nИтоговые результаты:")
        for result in sorted(all_results, key=lambda x: (x['task'])):
            print(f"{result['task']}:\"{result['text']}\"")

        print(f"\nВсего найдено {len(all_results)} результатов. Сохранено в {output_file}")
    except Exception as e:
        print(f"Ошибка сохранения результатов: {e}")

In [ ]:
if __name__ == "__main__":
    webres_dir = "school (1)"
    images_dir = "dataset/val/images"
    main(webres_dir, images_dir)